In [4]:
import numpy as np
import cv2
import os
import torch
import torchvision


import matplotlib.pyplot as plt
import plyfile

In [ ]:
# vertices: (|V|, 3), <x,y,z>
# triangles: (|F|, 3), vertex indices, 3 for each triangle
# sample_fn: how to select the color in space.

In [5]:
def spherical_sample_fn(x):
    '''
    Converts an x value to an rgb color based off of its normal

    Args:
        x: (N, 3)
    Returns:
        col: (N, 3), [0, 1] rgb
    '''
    x = x / np.linalg.norm(x)
    return x


def normuv(uv, H, W):
    """
    Normalize pixel coordinates to lie in [-1, 1]
    
    :param uv (..., 2) unnormalized pixel coordinates for HxW image
    """
    u = uv[..., 0] / H * 2.0 - 1.0
    v = uv[..., 1] / W * 2.0 - 1.0
    return torch.stack([u, v], dim=-1)


def unnormuv(uv, H, W):
    """
    Un-normalize pixel coordinates
    
    :param uv (..., 2) normalized pixel coordinates in [-1, 1]
    """
    u = (uv[..., 0] + 1.0) / 2.0 * H
    v = (uv[..., 1] + 1.0) / 2.0 * W
    return torch.stack([u, v], dim=-1)
    

def get_uv(H, W):
    """
    Get normalized uv coordinates for image
    :param height (int) source image height
    :param width (int) source image width
    :return uv (N, 2) pixel coordinates in [-1.0, 1.0]
    """
    yy, xx = torch.meshgrid(
        (torch.arange(H, dtype=torch.float32) + 0.5),
        (torch.arange(W, dtype=torch.float32) + 0.5),
        indexing='ij',
    )
    uv = torch.stack([xx, yy], dim=-1) # (H, W, 2)
    uv = normuv(uv, W, H) # (H, W, 2)
    return uv.view(H * W, 2)


def create_spherical_uv_sampling(im_shape):
    '''
    creates the sampling of directions from the origin around 360 space.
    Args:
        im_shape: (H, W)
    Returns:
        uv sampling map (H, W)
    '''
    H, W = im_shape
    phis_thetas = get_uv(H, W).numpy() * [np.pi, 2 * np.pi] # phi, theta
    uv_directions = np.column_stack(
        [
            np.sin(phis_thetas[:, 0]) * np.cos(phis_thetas[:, 1]),
            np.sin(phis_thetas[:, 0]) * np.sin(phis_thetas[:, 1]),
            np.cos(phis_thetas[:, 0]),
        ],
    )
    return uv_directions


In [6]:
def get_spherical_coords(X):
    # X is N x 3
    rad = np.linalg.norm(X, axis=1)
    # Inclination
    theta = np.arccos(X[:, 2] / rad)
    # Azimuth
    phi = np.arctan2(X[:, 1], X[:, 0])

    # Normalize both to be between [-1, 1]
    vv = (theta / np.pi) * 2 - 1
    uu = ((phi + np.pi) / (2*np.pi)) * 2 - 1
    # Return N x 2
    return np.stack([uu, vv],1)

In [70]:
def triangle_direction_intersection(tri, trg):
    '''
    Finds where an origin-centered ray going in direction trg intersects a triangle.
    Args:
        tri: 3 X 3 vertex locations. tri[0, :] is 0th vertex.
    Returns:
        alpha, beta, gamma
    '''
    p0 = np.copy(tri[0, :])
    # Don't normalize
    d1 = np.copy(tri[1, :]) - p0
    d2 = np.copy(tri[2, :]) - p0
    d = trg / np.linalg.norm(trg)

    mat = np.stack([d1, d2, d], axis=1)

    try:
      inv_mat = np.linalg.inv(mat)
    except np.linalg.LinAlgError:
      return False, 0
    
    a_b_mg = -1*np.matmul(inv_mat, p0)
    is_valid = (a_b_mg[0] >= 0) and (a_b_mg[1] >= 0) and ((a_b_mg[0] + a_b_mg[1]) <= 1) and (a_b_mg[2] < 0)
    if is_valid:
        return True, -a_b_mg[2]*d
    else:
        return False, 0

In [64]:
def mesh_centroid(vertices, faces):
    # mesh_volume = 0
    # temp = np.array([0.0,0.0,0.0])

    # for face in faces:
    #     v_a = vertices[int(face[0])]
    #     v_b = vertices[int(face[1])]
    #     v_c = vertices[int(face[2])]

    #     center = (v_a + v_b + v_c) / 4
    #     volume = np.dot(v_a, np.cross(v_b, v_c)) / 6.0
    #     mesh_volume += volume
    #     temp = center * volume

    # mesh_center = temp / mesh_volume
    # return mesh_center

    return np.average(vertices, axis=0)

def create_spherical_uv_map(H, W, vertices, faces, sample_fn=spherical_sample_fn):
    ray_dirs = create_spherical_uv_sampling((H, W)) # (H * W, 3)
    mesh_center = mesh_centroid(vertices, faces)

    normalized_vertices = vertices - mesh_center
    uv_map = np.zeros((H, W, 3))

    for face in faces:
        pass
    


    return uv_map

In [69]:
mesh_centroid(
    np.array(
        [
            [1, 1, 1],
            [1, 1, -1],
            [1, -1, -1],
            [1, -1, 1],
            [-1, 1, 1],
            [-1, 1, -1],
            [-1, -1, -1],
            [-1, -1, 1],
        ],
    ),
    np.array(
        [
            [0, 1, 2],
            [0, 2, 3],
            [1, 5, 2],
            [2, 5, 6],
            [3, 2, 6],
            [3, 6, 7],
            [4, 5, 1],
            [4, 1, 0],
            [4, 0, 3],
            [4, 3, 7],
            [4, 5, 6],
            [4, 6, 7],
        ]
    ),
)

array([0., 0., 0.])

In [13]:
get_uv(5, 5).shape

torch.Size([25, 2])

array([[-2.35619449, -4.71238898],
       [-0.78539816, -4.71238898],
       [ 0.78539816, -4.71238898],
       [ 2.35619449, -4.71238898],
       [-2.35619449, -1.57079633],
       [-0.78539816, -1.57079633],
       [ 0.78539816, -1.57079633],
       [ 2.35619449, -1.57079633],
       [-2.35619449,  1.57079633],
       [-0.78539816,  1.57079633],
       [ 0.78539816,  1.57079633],
       [ 2.35619449,  1.57079633],
       [-2.35619449,  4.71238898],
       [-0.78539816,  4.71238898],
       [ 0.78539816,  4.71238898],
       [ 2.35619449,  4.71238898]])

In [17]:
get_uv(4, 4)

tensor([[-0.7500, -0.7500],
        [-0.2500, -0.7500],
        [ 0.2500, -0.7500],
        [ 0.7500, -0.7500],
        [-0.7500, -0.2500],
        [-0.2500, -0.2500],
        [ 0.2500, -0.2500],
        [ 0.7500, -0.2500],
        [-0.7500,  0.2500],
        [-0.2500,  0.2500],
        [ 0.2500,  0.2500],
        [ 0.7500,  0.2500],
        [-0.7500,  0.7500],
        [-0.2500,  0.7500],
        [ 0.2500,  0.7500],
        [ 0.7500,  0.7500]])

In [26]:
phis_thetas = get_uv(4, 4).numpy() * [np.pi, 2 * np.pi]

In [38]:
np.column_stack(
    [
        np.sin(phis_thetas[:, 0]) * np.cos(phis_thetas[:, 1]),
        np.sin(phis_thetas[:, 0]) * np.sin(phis_thetas[:, 1]),
        np.cos(phis_thetas[:, 0]),
    ],
).shape

(16, 3)

In [28]:
np.sin(phis_thetas[:, 0]) * np.cos(phis_thetas[:, 1])

array([ 1.29893408e-16,  1.29893408e-16, -1.29893408e-16, -1.29893408e-16,
       -4.32978028e-17, -4.32978028e-17,  4.32978028e-17,  4.32978028e-17,
       -4.32978028e-17, -4.32978028e-17,  4.32978028e-17,  4.32978028e-17,
        1.29893408e-16,  1.29893408e-16, -1.29893408e-16, -1.29893408e-16])

In [7]:
np.hstack([np.ones((5, 1)), np.zeros((5, 1))])

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [32]:
def numpy_to_ply(vertices, faces, vert_colors=None, filename=None):
    '''
    Converts numpy arrays to PLY file, which can be used
    in blender or converted into an obj file.
    Args:
        # vertices: (|V|, 3), <x,y,z>
        # faces: (|F|, 3), vertex indices, 3 for each triangle
        # vert_colors (optional): (|V|, 3) RGB, [0, 1] range
        # filename: string. if not None, write to given filepath.
    Returns:
        PlyData obj
    '''
    assert vertices.shape[1] == 3 and faces.shape[1] == 3
    vertices = np.array(
        [tuple(vertices[i]) for i in range(vertices.shape[0])],
        dtype=[('x', 'f4'), ('y', 'f4'), ('z', 'f4')],
    )
    faces = np.array(
        [tuple([faces[i]]) for i in range(faces.shape[0])],
        dtype=[('vertex_indices', 'i4', (3,))],
    )
    ply_data = plyfile.PlyData(
        [
            plyfile.PlyElement.describe(
                vertices, 
                'vertex',
                comments=['model vertices'],
            ),
            plyfile.PlyElement.describe(
                faces,
                'face',
            ),
        ],
    )
    if filename:
        ply_data.write(filename)
    
    return ply_data


In [33]:
unit_box_vertices = np.array(
    [
        [1, 1, 1],
        [1, 1, -1],
        [1, -1, -1],
        [1, -1, 1],
        [-1, 1, 1],
        [-1, 1, -1],
        [-1, -1, -1],
        [-1, -1, 1],
    ],
)

unit_box_faces = np.array(
    [
        [0, 1, 2],
        [0, 2, 3],
        [1, 5, 2],
        [2, 5, 6],
        [3, 2, 6],
        [3, 6, 7],
        [4, 5, 1],
        [4, 1, 0],
        [4, 0, 3],
        [4, 3, 7],
        [4, 5, 6],
        [4, 6, 7],
    ]
)


In [35]:
numpy_to_ply(unit_box_vertices, unit_box_faces, filename="test.ply")

PlyData((PlyElement('vertex', (PlyProperty('x', 'float'), PlyProperty('y', 'float'), PlyProperty('z', 'float')), count=8, comments=['model vertices']), PlyElement('face', (PlyListProperty('vertex_indices', 'uchar', 'int'),), count=12, comments=[])), text=False, byte_order='<', comments=[], obj_info=[])

In [23]:
np.array(
    [tuple(unit_box_vertices[i]) for i in range(unit_box_vertices.shape[0])],
    dtype=[('x', 'f4'), ('y', 'f4'), ('z', 'f4')],
)

array([( 1.,  1.,  1.), ( 1.,  1., -1.), ( 1., -1., -1.), ( 1., -1.,  1.),
       (-1.,  1.,  1.), (-1.,  1., -1.), (-1., -1., -1.), (-1., -1.,  1.)],
      dtype=[('x', '<f4'), ('y', '<f4'), ('z', '<f4')])

In [25]:
np.array(
    [tuple([unit_box_faces[i]]) for i in range(unit_box_faces.shape[0])],
    dtype=[('vertex_indices', 'i4', (3,))],
)

array([([0, 1, 2],), ([0, 2, 3],), ([1, 5, 2],), ([2, 5, 6],),
       ([3, 2, 6],), ([3, 6, 7],), ([4, 5, 1],), ([4, 1, 0],),
       ([4, 0, 3],), ([4, 3, 7],), ([4, 5, 6],), ([4, 6, 7],)],
      dtype=[('vertex_indices', '<i4', (3,))])